<a href="https://colab.research.google.com/github/Bruno-GSilva/price-prediction/blob/main/model_1/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model 1

### Mount Drive to access data

In [112]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Imports

In [113]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

## Data Loader

In [114]:
class PriceDataset(Dataset):

  def __init__(self, transform=None):

    #Read CSV file
    df = pd.read_csv("/content/drive/MyDrive/data/PETR4.SA.csv", parse_dates=[0], infer_datetime_format=True)
    df.dropna()

    #Add collumn indicating when there is a time skip > 1 day
    ttime_skip = [ 0 if index == 0 else 1 if (row['Date'] - df['Date'][index - 1]).days > 1 else 0 for index, row in df.iterrows() ]
    df['Skip dates'] = time_skip

    #Filter usefull collumns
    df_filter = df.filter(['Open', 'High', 'Low', 'Close', 'Skip dates'])
    data = df_filter.values

    #Scale input
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)

    #Create samples
    x_samples = []
    y_samples = []
    for i in range (60, len(data)):
      x_samples.append(data[i-60 : i])
      y_samples.append(data[i,3])

    x_samples, y_samples = np.array(x_samples), np.array(y_samples)

    #Transform int tensors
    self.x_data = torch.from_numpy(x_samples)
    self.y_data = torch.from_numpy(y_samples)
    self.n_data = y_samples.shape[0]

    #Set transform
    self.transform = transform

  def __getitem__(self, index):
    sample = self.x_data[index], self.y_data[index]
    if self.transform:
            sample = self.transform(sample)
    return sample

  def __len__(self):
    return self.n_data

## Model

In [115]:
# Fully connected neural network 
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.hidden_layers = []
        for i in range(num_layers - 1):
          self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))
        self.output_layer = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)

        for i in range(len(hidden_layers)):
          out = self.hidden_layers[i](out)
          out = self.relu(out)

        out = self.output_layer(out)
        # no activation and no softmax at the end
        return out

## Create DataLoader

In [116]:
class ToLinearTensor:
  def __call__(self, sample):
    inputs, targets = sample
    return inputs.view(inputs.shape[0]*inputs.shape[1]), targets


In [117]:
dataset = PriceDataset(transform=ToLinearTensor())
train_loader = DataLoader(dataset=dataset,
                          batch_size=4,
                          shuffle=True,
                          num_workers=2)